In [4]:
from tweepy.streaming import StreamListener

from tweepy import OAuthHandler
from tweepy import API

from tweepy import Stream

import time
import json
import sys


In [ ]:


#class SListener(StreamListener):
    #def __init__(self, api = None):
        #self.output = open('tweets_%s.json' % 
                          #time.strftime('%Y%m%d-%H%M%S'), 'w')
        #self.api = api or API()

In [ ]:
# Object, whicht tells how to handle incoming data

class SListener(StreamListener):
    def __init__(self, api = None, fprefix = 'streamer'):
        self.api = api or API()
        self.counter = 0
        self.fprefix = fprefix
        self.output  = open('%s_%s.json' % (self.fprefix, time.strftime('%Y%m%d-%H%M%S')), 'w')


    def on_data(self, data):
        if  'in_reply_to_status' in data:
            self.on_status(data)
        elif 'delete' in data:
            delete = json.loads(data)['delete']['status']
            if self.on_delete(delete['id'], delete['user_id']) is False:
                return False
        elif 'limit' in data:
            if self.on_limit(json.loads(data)['limit']['track']) is False:
                return False
        elif 'warning' in data:
            warning = json.loads(data)['warnings']
            print("WARNING: %s" % warning['message'])
            return


    def on_status(self, status):
        self.output.write(status)
        self.counter += 1
        if self.counter >= 20000:
            self.output.close()
            self.output  = open('%s_%s.json' % (self.fprefix, time.strftime('%Y%m%d-%H%M%S')), 'w')
            self.counter = 0
        return


    def on_delete(self, status_id, user_id):
        print("Delete notice")
        return


    def on_limit(self, track):
        print("WARNING: Limitation notice received, tweets missed: %d" % track)
        return


    def on_error(self, status_code):
        print('Encountered error with status code:', status_code)
        return 


    def on_timeout(self):
        print("Timeout, sleeping for 60 seconds...")
        time.sleep(60)
        return 

In [ ]:
# Authentication protocol, Consumer key authentication
auth = OAuthHandler(consumer_key, consumer_secret)
# Setting the access token and access token secret, Access key authentication
auth.set_access_token(access_token, access_token_secret)
# Pass the auth object to the tweepy API object
api = API (auth)

Collecting data on keywords

Now that we've set up the authentication, we can begin to collect Twitter data. Recall that with the Streaming API, we will be collecting real-time Twitter data based on either a sample or filtered by a keyword.

In our example, we will collect data on any tweet mentioning #rstats or #python in the tweet text, username, or user description with the filter endpoint.

The SListener module has already been defined and imported for you. You can find the full code for this module here.

In [ ]:
# Set up words to track
keywords_to_track = ['#rstats', '#python']

In [ ]:
# Instatiating Slistener object
listen = SListener(api)
# Instantiating the stream object
stream = Stream(auth, listen)
#We call the sample method to begin collecting data
#stream.sample()


In [ ]:
# Begin collecting data
stream.filter(track = keywords_to_track)